# Sentiment Analysis Using Various Approaches

## Lexicon-based approach 
- Unsupervised learning
- Based on calculating sentiment scores of words in a document from lexicons.
- Each word's sentiment is determined, and the scores are combined to calculate the overall sentiment of the sentence. 
- A lexicon is a dictionary that contains a collection of words that is categorized as positive, negative, and neutral by experts. Their scores can change over time. 

In [1]:
import numpy as np 
import pandas as pd
import json
import time
import re

# Text cleaning
from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# NLTK Bing Liu Lexicon 
import nltk
# nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize 

# VADER 
import nltk.corpus
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## Loading a subset of reviews and meta data

In [2]:
n = 1 
total_rows = 0

def process_chunks(file, chunksize = 1000):

    # Setting as global variables
    global n, total_rows  
    
    chunks = pd.read_json(file, lines=True, chunksize = chunksize)
    dfs = []  
    n_chunks = 0

    for chunk in chunks:
        dfs.append(chunk)
        n_chunks += 1  # Count the number of chunks processed
        print(len(chunk), " rows added")
        n += 1 
        total_rows += len(chunk)
        if n_chunks >= 10:  # Process only the first 5 chunks
            break  
            
    print("Done")
    print("Total rows:", total_rows)
    return pd.concat(dfs, ignore_index=True)

In [3]:
reviews = "../data/Home_and_Kitchen.jsonl"
meta = "../data/meta_Home_and_Kitchen.jsonl"

start = time.process_time()

reviews_subset = process_chunks(reviews)

end = time.process_time()
elapsed_time = end - start
print('Created a subset of the reviews dataset')
print('Execution time:', elapsed_time, 'seconds')

print('--------------')
start = time.process_time()

meta_subset = process_chunks(meta)

end = time.process_time()
elapsed_time = end - start
print('Created a subset of the meta dataset')
print('Execution time:', elapsed_time, 'seconds')

1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
Done
Total rows: 10000
Created a subset of the reviews dataset
Execution time: 0.390625 seconds
--------------
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
1000  rows added
Done
Total rows: 20000
Created a subset of the meta dataset
Execution time: 1.359375 seconds


In [4]:
reviews_subset.head(3)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,1,Received Used & scratched item! Purchased new!,Livid. Once again received an obviously used ...,[],B007WQ9YNO,B09XWYG6X1,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2023-02-26 01:03:29.298,1,True
1,5,Excellent for moving & storage & floods!,I purchased these for multiple reasons. The ma...,[],B09H2VJW6K,B0BXDLF8TW,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-12-26 08:30:10.846,0,True
2,2,Lid very loose- needs a gasket imo. Small base.,[[VIDEOID:c87e962bc893a948856b0f1b285ce6cc]] I...,[{'small_image_url': 'https://m.media-amazon.c...,B07RL297VR,B09G2PW8ZG,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-05-25 02:54:56.788,0,True


In [5]:
reviews_subset.columns

Index(['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase'],
      dtype='object')

## Text Cleaning - spaCy

In [6]:
import spacy 
nlp = spacy.load('en_core_web_sm')

In [39]:
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Update stopwords
include_stopwords = {'would', 'I', 'not'}
exclude_stopwords = {'i', 'well'}

stop_words |= include_stopwords
stop_words -= exclude_stopwords

def clean_data(doc):
    doc = doc.lower()
    doc = nlp(doc)
    # Lemmatize words 
    lemmas = [token.lemma_ for token in doc]
    # Removing non-alphabetic characters and stopwords
    tokens = [lemma for lemma in lemmas if lemma.isalpha() and lemma not in stop_words]
    cleaned_doc = " ".join(tokens)
    
    return cleaned_doc

In [40]:
stop_words

{"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'I',
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from',


In [41]:
lemmatizer = WordNetLemmatizer()
stop__words = set(stopwords.words('english'))

# Update stopwords
include_stopwords = {'would', 'I', 'not'}
exclude_stopwords = {'i'}

stop__words |= include_stopwords
stop__words -= exclude_stopwords

def preprocess_text(text):
    text = text.lower()  # Lowercase text
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove punctuation and numbers
    tokens = text.split()  # Create tokens 
    clean_tokens = [lemmatizer.lemmatize(t) for t in tokens if t not in stop__words]  # Lemmatize and remove stop words
    clean_text = " ".join(clean_tokens)  # Join clean tokens
    clean_text = " ".join(clean_text.split())  # Remove extra spaces, tabs, and new lines
    
    return clean_text

In [42]:
stop__words

{'I',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [43]:
cleaned_text = reviews_subset.copy()
cleaned_text['spacy_text'] = cleaned_text['text'].apply(clean_data)
cleaned_text['nltk_text'] = cleaned_text['text'].apply(preprocess_text)

In [44]:
cleaned_text.head(3)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,spacy_text,nltk_text
0,1,Received Used & scratched item! Purchased new!,Livid. Once again received an obviously used ...,[],B007WQ9YNO,B09XWYG6X1,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2023-02-26 01:03:29.298,1,True,livid receive obviously use item food scratch ...,livid received obviously used item food scratc...
1,5,Excellent for moving & storage & floods!,I purchased these for multiple reasons. The ma...,[],B09H2VJW6K,B0BXDLF8TW,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-12-26 08:30:10.846,0,True,purchase multiple reason main reason bc apt fl...,i purchased multiple reason main reason i movi...
2,2,Lid very loose- needs a gasket imo. Small base.,[[VIDEOID:c87e962bc893a948856b0f1b285ce6cc]] I...,[{'small_image_url': 'https://m.media-amazon.c...,B07RL297VR,B09G2PW8ZG,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-05-25 02:54:56.788,0,True,videoid want love bc previously buy matching t...,videoidcebcabfbcecc i wanted love bc i previou...


## Bing Liu Lexicon

The Bing Liu lexicon has a total of 6, 786 words with 2,005 classified as positive and 4,781 as negative. CLassification is binary (positive or negative).

In [33]:
print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))
print('Examples of positive words:', opinion_lexicon.positive()[:10])
print('Examples of negative words:', opinion_lexicon.negative()[:10])

Total number of words in opinion lexicon 6789
Examples of positive words: ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words: ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


In [34]:
pos_score = 1
neg_score = -1
word_dict = {}

# Adding the positive words to the dictionary
for word in opinion_lexicon.positive():
    word_dict[word] = pos_score 

# Adding the negative words to the dictionary 
for word in opinion_lexicon.negative():
    word_dict[word] = neg_score 

def bing_liu_score(text):
    sentiment_score = 0 
    bag_of_words = word_tokenize(text.lower())

    # Check if bag_of_words is empty
    if bag_of_words: 
        for word in bag_of_words: 
            if word in word_dict: 
                sentiment_score += word_dict[word]
        return sentiment_score / len(bag_of_words)
    else: 
        return 0

In [45]:
cleaned_text['Bing_Liu_score'] = cleaned_text['text'].apply(bing_liu_score)
cleaned_text['Bing_Liu_spaCy'] = cleaned_text['spacy_text'].apply(bing_liu_score)
cleaned_text['Bing_Liu_nltk'] = cleaned_text['nltk_text'].apply(bing_liu_score)
cleaned_text.head(5)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,spacy_text,nltk_text,Bing_Liu_score,Bing_Liu_spaCy,Bing_Liu_nltk
0,1,Received Used & scratched item! Purchased new!,Livid. Once again received an obviously used ...,[],B007WQ9YNO,B09XWYG6X1,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2023-02-26 01:03:29.298,1,True,livid receive obviously use item food scratch ...,livid received obviously used item food scratc...,-0.085714,-0.200000,-0.187500
1,5,Excellent for moving & storage & floods!,I purchased these for multiple reasons. The ma...,[],B09H2VJW6K,B0BXDLF8TW,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-12-26 08:30:10.846,0,True,purchase multiple reason main reason bc apt fl...,i purchased multiple reason main reason i movi...,-0.006568,-0.027650,-0.019868
2,2,Lid very loose- needs a gasket imo. Small base.,[[VIDEOID:c87e962bc893a948856b0f1b285ce6cc]] I...,[{'small_image_url': 'https://m.media-amazon.c...,B07RL297VR,B09G2PW8ZG,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-05-25 02:54:56.788,0,True,videoid want love bc previously buy matching t...,videoidcebcabfbcecc i wanted love bc i previou...,-0.008287,-0.031496,-0.016949
3,5,Best purchase ever!,If you live at a higher elevation like me (5k ...,[{'small_image_url': 'https://m.media-amazon.c...,B09CQF4SWV,B08CSZDXZY,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-05-06 16:38:16.178,0,True,live high elevation like colorado know buzzer ...,live higher elevation like k colorado know buz...,0.006316,-0.044304,0.009615
4,5,Excellent for yarn!,I use these to store yarn. They easily hold 12...,[{'small_image_url': 'https://images-na.ssl-im...,B003U6A3EY,B0C6V27S6N,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2020-05-20 00:28:45.940,1,True,use store yarn easily hold ounce bernat pipsqu...,i use store yarn easily hold ounce bernat pips...,0.035294,0.062500,0.075000


In [46]:
cleaned_text[['rating', 'text', 'Bing_Liu_score', 'Bing_Liu_spaCy', 'Bing_Liu_nltk']].sample(5)

,rating,text,Bing_Liu_score,Bing_Liu_spaCy,Bing_Liu_nltk
37,3,It works okay. But I have to tinker with it al...,0.097561,0.357143,0.181818
5759,5,I ordered this door decoration for my elderly ...,0.051020,0.131579,0.102041
5387,2,These are smaller than teaspoons that come in ...,0.000000,0.000000,0.000000
9167,5,The price was right I was able to scrape out ...,0.058824,0.250000,0.150000
2808,3,It's nice for tight spaces but quite flimsy,0.000000,0.000000,0.000000


In [37]:
# Look for scores of zeroes in Bing_Liu_spaCy and Bing_Liu_nltk columns 
cleaned_text[cleaned_text['Bing_Liu_spaCy'] == 0]

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,spacy_text,nltk_text,Bing_Liu_score,Bing_Liu_spaCy,Bing_Liu_nltk
15,5,Perfect mattress,"OK, we bought this mattress for our guest room...",[],B0777K9RGX,B0BPBLYF85,AGGZ357AO26RQZVRLGU4D4N52DZQ,2020-01-06 02:16:33.373,0,True,ok buy mattress guest room wish mattress serio...,ok bought mattress guest room im wishing mattr...,0.000000,0.0,0.000000
25,4,Nice sparkly touch,Nice quality. A little pricey for the amount b...,[],B01I3Q6BHS,B01I3Q6BHS,AGKASBHYZPGTEPO6LWZPVJWB2BVA,2017-01-30 15:35:05.000,0,True,nice quality little pricey reusable desire,nice quality little pricey amount also reusabl...,0.000000,0.0,0.000000
35,5,yay,happily loved by our 1 year old. she makes fu...,[],B00563XRYM,B00563XRYM,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,2013-11-20 01:54:13.000,0,True,happily love year old funny noise think funny ...,happily loved year old make funny noise think ...,0.000000,0.0,0.000000
52,3,I love the colors & systles of the mugs,"I love the colors & systles of the mugs, but t...",[],B004O39RJ4,B004O39RJ4,AGXVBIUFLFGMVLATYXHJYL4A5Q7Q,2015-04-06 13:13:55.000,0,True,love color systle mug small size anticipate gr...,i love color systles mug smaller size i antici...,0.000000,0.0,0.000000
60,5,It peels garlic,It does the job.,[],B077MNPF15,B077MNPF15,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2019-07-23 04:28:05.311,0,True,job,job,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9976,5,Does the job!,Just what I was looking for.,[],B0765BTD13,B0BWJ277SP,AE57MRF2R2ALCC6H5WQLFKT7KSSA,2021-03-05 15:23:36.385,0,True,look,i looking,0.000000,0.0,0.000000
9979,1,IT INTERFERS WITH THE STEAMER BUTTON,It is lightweight and deep enough to steam ent...,[],B079CLQJ5Z,B079CLQJ5Z,AE57MRF2R2ALCC6H5WQLFKT7KSSA,2019-10-05 16:57:37.457,2,True,lightweight deep steam entire head broccoli ca...,lightweight deep enough steam entire head broc...,0.018182,0.0,0.034483
9987,1,Not comfortable,"Uncomfortable, not cool, and disappointed. Eas...",[],B07MY34QH7,B0BLXVK6QX,AFPHKIJFGIU4G4POXRFCEF5RJJHA,2020-12-05 16:13:34.579,1,True,uncomfortable cool disappoint easy return policy,uncomfortable cool disappointed easy return po...,0.000000,0.0,0.000000
9991,5,Nice cups,"Gave as a gift, well received",[],B079L1Q2KK,B079L1Q2KK,AFPHKIJFGIU4G4POXRFCEF5RJJHA,2020-12-05 15:12:34.299,0,True,gift receive,gave gift well received,0.142857,0.0,0.250000


In [47]:
i = 3
print(f'Title: {cleaned_text.loc[i,"title"]}\n')

print(f'Text: {cleaned_text.loc[i,"text"]}\n')

print(f'Text: {cleaned_text.loc[i,"spacy_text"]}\n')

print(f'Text: {cleaned_text.loc[i,"nltk_text"]}')

Title: Best purchase ever!

Text: If you live at a higher elevation like me (5k Colorado) just know that after the buzzer beeps you might wanna leave the hard boiled eggs inside the steamed cooker for another minute or two if you want the egg cooked all the way through. I also add a bit more water than the fill line says, but again that’s bc of my elevation & it evaporates quickly.  It’s not any quicker than I used to make them, but it’s a lot less water. I’m disabled & have a TBI rn so the auto-cutoff was a no-brained for me. I bought the blue bc it was cheapest.  My service 🐶  🐶 love hard boiled eggs too so we are enjoying this new addition to our kitchen.  My only regret is that I did not buy it sooner & that Amazon never has the color I want for the cheapest price, but I can live with the bb blue.  We have used it 3x & so far it’s working okay. Oh- don’t place it under upper cabinets as it has a vent hole & it will vent & it will make the cabinets sweat.  Give it some room to breat

In [22]:
# Calculate mean sentiment score for each rating category
mean_scores = cleaned_text.groupby('rating').agg({
    'Bing_Liu_score':'mean',
    'Bing_Liu_spaCy': 'mean',
    'Bing_Liu_nltk': 'mean'
}).reset_index()

print(mean_scores)

   rating  Bing_Liu_score  Bing_Liu_spaCy  Bing_Liu_nltk
0       1       -0.031370       -0.074885      -0.051852
1       2        0.003011       -0.003263       0.010138
2       3        0.031123        0.057409       0.065607
3       4        0.066978        0.140987       0.137529
4       5        0.129455        0.249908       0.236131


In [23]:
# Calculate max sentiment score for each rating category
max_scores = cleaned_text.groupby('rating').agg({
    'Bing_Liu_score':'max',
    'Bing_Liu_spaCy': 'max',
    'Bing_Liu_nltk': 'max'
}).reset_index()

print(max_scores)

   rating  Bing_Liu_score  Bing_Liu_spaCy  Bing_Liu_nltk
0       1        0.333333             1.0            1.0
1       2        0.250000             1.0            1.0
2       3        1.000000             1.0            1.0
3       4        1.000000             1.0            1.0
4       5        1.000000             1.0            1.0


In [24]:
# Calculate min sentiment score for each rating category
min_scores = cleaned_text.groupby('rating').agg({
    'Bing_Liu_score':'min',
    'Bing_Liu_spaCy': 'min',
    'Bing_Liu_nltk': 'min'
}).reset_index()

print(min_scores)

   rating  Bing_Liu_score  Bing_Liu_spaCy  Bing_Liu_nltk
0       1       -1.000000            -1.0      -1.000000
1       2       -0.666667            -1.0      -0.666667
2       3       -0.500000            -1.0      -0.500000
3       4       -1.000000            -1.0      -1.000000
4       5       -0.500000            -1.0      -1.000000


## VADER Lexicon
Rule-based lexicon. 
9,000 features with scales of [-4] Extremely Negative to [4] Extremely Positive with [0] for Neutral or Neither. 